In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline

classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device=0)
classifier("I love this!")


Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528676815330982},
  {'label': 'neutral', 'score': 0.005764591973274946},
  {'label': 'anger', 'score': 0.004419781267642975},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385195800103247}]]

In [3]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [4]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548433899879456},
  {'label': 'neutral', 'score': 0.16985109448432922},
  {'label': 'sadness', 'score': 0.11640793830156326},
  {'label': 'surprise', 'score': 0.020700635388493538},
  {'label': 'disgust', 'score': 0.019100651144981384},
  {'label': 'joy', 'score': 0.015161151066422462},
  {'label': 'anger', 'score': 0.003935144282877445}]]

In [5]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296026945114136},
  {'label': 'neutral', 'score': 0.14038574695587158},
  {'label': 'fear', 'score': 0.06816213577985764},
  {'label': 'joy', 'score': 0.04794243350625038},
  {'label': 'anger', 'score': 0.009156356565654278},
  {'label': 'disgust', 'score': 0.002628471702337265},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937047362327576},
  {'label': 'disgust', 'score': 0.2735918462276459},
  {'label': 'joy', 'score': 0.109082892537117},
  {'label': 'sadness', 'score': 0.09362728148698807},
  {'label': 'anger', 'score': 0.04047832638025284},
  {'label': 'surprise', 'score': 0.026970166712999344},
  {'label': 'fear', 'score': 0.0068790502846241}],
 [{'label': 'neutral', 'score': 0.6462167501449585},
  {'label': 'sadness', 'score': 0.24273262917995453},
  {'label': 'disgust', 'score': 0.04342260956764221},
  {'label': 'surprise', 'score': 0.028300534933805466},
  {'label': 'joy', 'score': 0.014211485162

In [6]:
sentence = books["description"][0].split(".")
prediction = classifier(sentence)


In [7]:
sentence[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [8]:
prediction[0]

[{'label': 'surprise', 'score': 0.7296026945114136},
 {'label': 'neutral', 'score': 0.14038574695587158},
 {'label': 'fear', 'score': 0.06816213577985764},
 {'label': 'joy', 'score': 0.04794243350625038},
 {'label': 'anger', 'score': 0.009156356565654278},
 {'label': 'disgust', 'score': 0.002628471702337265},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [9]:
# sorted filter map reduce 传入函数
sorted(prediction[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156356565654278},
 {'label': 'disgust', 'score': 0.002628471702337265},
 {'label': 'fear', 'score': 0.06816213577985764},
 {'label': 'joy', 'score': 0.04794243350625038},
 {'label': 'neutral', 'score': 0.14038574695587158},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296026945114136}]

In [21]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "neutral", "sadness", "surprise", "joy"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


# predictions 是一个数组，存放每一句话的情感分类
# 现在把predictions拆开，根据label排序
# 便利一个prediction里的字典
# 最后返回字典，每个label的最大值
def calculate_emotion_scores(predictions):
    per_emotion_score = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_score[label].append(sorted_prediction[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_score.items()}

In [22]:
emotion_scores

{'anger': [],
 'disgust': [],
 'fear': [],
 'neutral': [],
 'sadness': [],
 'surprise': [],
 'joy': []}

In [23]:
from tqdm import tqdm
emotion_labels = ["anger", "disgust", "fear", "neutral", "sadness", "surprise", "joy"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_score[label])

100%|██████████| 5197/5197 [03:21<00:00, 25.77it/s]


In [24]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df['isbn13'] = isbn

In [25]:
emotion_df

,anger,disgust,fear,neutral,sadness,surprise,joy,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646217,0.967157,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255169,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765,9788185300535


In [26]:
books = pd.merge(books, emotion_df, on="isbn13")

In [27]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,sadness_x,surprise_x,joy_x,anger_y,disgust_y,fear_y,neutral_y,sadness_y,surprise_y,joy_y
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,0.140386,0.002122,0.729603,0.064134,0.273592,0.928168,0.932797,0.646217,0.967157,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,0.217761,0.004509,0.060646,0.612619,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,0.042176,0.010860,0.009796,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,0.732685,0.020988,0.006147,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,0.272613,0.475881,0.012260,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,0.010511,0.857255,0.008218,0.148209,0.030643,0.919165,0.255169,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,0.338892,0.005487,0.227765,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,0.329755,0.010788,0.008101,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,0.648011,0.017372,0.032275,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765


In [28]:
books.to_csv("books_with_emotions.csv", index=False)